In [1]:
# Setup

# img_directory = '/Users/rwilliams/Desktop/celeba/training'
img_directory = '/home/ec2-user/training-data/img_align_celeba'
model_save_path = '/home/ec2-user/tf-checkpoints/vae-celeba/checkpoint.ckpt'
outputs_directory = '/home/ec2-user/outputs/vae-celeba'
log_directory = '/home/ec2-user/tf-logs/vae-celeba'
batch_size = 64
training_set_size = 5000
img_size = 64

# for adam optimizer
learning_rate = 2e-4
learning_beta1 = 0.9

In [2]:
# import packages for jupyter
# import matplotlib.pyplot as plt
# %matplotlib inline

In [3]:
# imports

import numpy as np
import scipy as sp
import os
from utils import imshow, resize_crop, load_img

In [4]:
# load training data

training = np.array([resize_crop(load_img(i+1, img_directory), (img_size, img_size)) for i in range(training_set_size)])

/home/ec2-user/.conda/envs/keras/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [5]:
# Create model and load weights

import tensorflow as tf
from autoencoder import Autoencoder
tf.reset_default_graph()

vae = Autoencoder(img_shape=(img_size, img_size, 3))

X = tf.placeholder(tf.float32, [None, img_size, img_size, 3])

# flags to pass to networks to set batch normalization layers
# as trainable or not
batch_trainable = tf.placeholder(tf.bool)

encoder = vae.encoder(X, batch_trainable)
decoder = vae.decoder(encoder, batch_trainable)

xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=X, logits=vae.logits)
latent_loss = vae.latent_loss()
reconstruction_loss = tf.reduce_mean(xentropy)
loss = reconstruction_loss + latent_loss

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=learning_beta1)

with tf.control_dependencies(update_ops):
    training_op = optimizer.minimize(loss)

saver = tf.train.Saver()

In [6]:
# create or restore session

sess = tf.InteractiveSession()
try:
    print('trying to restore session')
    saver.restore(sess, model_save_path)
    print('restored session')
except:
    print('failed to restore session, creating a new one')
    tf.global_variables_initializer().run()

# write logs for tensorboard
writer = tf.summary.FileWriter(log_directory, sess.graph)

trying to restore session
INFO:tensorflow:Restoring parameters from /home/ec2-user/tf-checkpoints/vae-celeba/checkpoint.ckpt
failed to restore session, creating a new one


In [ ]:
# collect data for tensorboard
tf.summary.scalar('latent_loss', latent_loss)
tf.summary.scalar('reconstruction_loss', reconstruction_loss)
tf.summary.scalar('total_loss', loss)
merged_summary = tf.summary.merge_all()

In [ ]:
import math
epochs = 10000
batches = int(float(training_set_size) / batch_size) 

img_idx = 0
print('training', flush=True)
for epoch in range(epochs):
    
    print ('epoch %s ' % epoch, end='', flush=True)
    for batch in range(batches):
        print('.', end='', flush=True)
        feed = training[batch*batch_size:(batch+1)*batch_size]
        sess.run(training_op, feed_dict={X: feed, batch_trainable: True})
        
    if (epoch % 1 == 0):
        print('saving session', flush=True)
        saver.save(sess, model_save_path)
        
        xfeed = training[:batch_size]
        summary = merged_summary.eval(feed_dict={X: xfeed, batch_trainable: False})
        writer.add_summary(summary, epoch) 

        example = decoder.eval(feed_dict={X: training[:1], batch_trainable: False})
        img_save_path = os.path.join(outputs_directory, '%06d.jpg' % img_idx)
        img_idx += 1
        sp.misc.imsave(img_save_path, example[0])

training
epoch 0 ..............................................................................saving session
epoch 1 ..............................................................................saving session
epoch 2 ..............................................................................saving session
epoch 3 ..............................................................................saving session
epoch 4 ..............................................................................saving session
epoch 5 ..............................................................................saving session
epoch 6 ..............................................................................saving session
epoch 7 ..............................................................................saving session
epoch 8 ..............................................................................saving session
epoch 9 ..........................................................................

epoch 80 ..............................................................................saving session
epoch 81 ..............................................................................saving session
epoch 82 ..............................................................................saving session
epoch 83 ..............................................................................saving session
epoch 84 ..............................................................................saving session
epoch 85 ..............................................................................saving session
epoch 86 ..............................................................................saving session
epoch 87 ..............................................................................saving session
epoch 88 ..............................................................................saving session
epoch 89 .........................................................................